In [10]:
import pandas as pd

def calculate_positional_weights(tasks):
    times = {task['Task']: task['Time Required'] for task in tasks}
    precedences = {task['Task']: task['Immediate Predecessors'] for task in tasks}
    weights = {}

    for task in tasks:
        task_id = task['Task']
        weight = times[task_id]
        successors = set()

        def add_successors(t):
            nonlocal weight  # Declare 'weight' as nonlocal to modify it within nested function
            for succ in [k for k, v in precedences.items() if t in v]:
                if succ not in successors:
                    successors.add(succ)
                    weight += times[succ]
                    add_successors(succ)

        add_successors(task_id)
        weights[task_id] = weight
    return weights

def assign_tasks_to_stations(weights, tasks, times, cycle_time):
    sorted_tasks = sorted(tasks, key=lambda x: weights[x['Task']], reverse=True)
    task_precedences = {task['Task']: set(task['Immediate Predecessors']) for task in tasks}
    task_assigned = set()
    stations = []
    current_station = []
    current_time = 0

    while sorted_tasks:
        task_added = False
        for task in sorted_tasks:
            task_id = task['Task']
            task_time = times[task_id]
            if task_precedences[task_id].issubset(task_assigned) and current_time + task_time <= cycle_time:
                current_station.append(task_id)
                current_time += task_time
                task_assigned.add(task_id)
                sorted_tasks.remove(task)
                task_added = True
                break

        if not task_added:  # If no task was added, move to the next station
            stations.append((current_station, cycle_time - current_time))
            current_station = []
            current_time = 0
    
    if current_station:  # Add remaining station if there is any task left unassigned
        stations.append((current_station, cycle_time - current_time))
    
    return stations

def format_output_as_dataframe(stations):
    # Creating a DataFrame from the station data
    data = {
        'Station': [i+1 for i in range(len(stations))],
        'Tasks': [', '.join(station[0]) for station in stations],
        'Idle time': [station[1] for station in stations]
    }
    df = pd.DataFrame(data)
    return df

tasks = [
    {'Task': '1', 'Time Required': 12, 'Immediate Predecessors': []},
    {'Task': '2', 'Time Required': 6, 'Immediate Predecessors': ['1']},
    {'Task': '3', 'Time Required': 6, 'Immediate Predecessors': ['2']},
    {'Task': '4', 'Time Required': 2, 'Immediate Predecessors': ['2']},
    {'Task': '5', 'Time Required': 2, 'Immediate Predecessors': ['2']},
    {'Task': '6', 'Time Required': 12, 'Immediate Predecessors': ['2']},
    {'Task': '7', 'Time Required': 7, 'Immediate Predecessors': ['3', '4']},
    {'Task': '8', 'Time Required': 5, 'Immediate Predecessors': ['7']},
    {'Task': '9', 'Time Required': 1, 'Immediate Predecessors': ['5']},
    {'Task': '10', 'Time Required': 4, 'Immediate Predecessors': ['9', '6']},
    {'Task': '11', 'Time Required': 6, 'Immediate Predecessors': ['8', '10']},
    {'Task': '12', 'Time Required': 7, 'Immediate Predecessors': ['11']}
]

cycle_time = 13

# Calculate positional weights
weights = calculate_positional_weights(tasks)
times = {task['Task']: task['Time Required'] for task in tasks}

# Assign tasks to stations considering the cycle time
stations = assign_tasks_to_stations(weights, tasks, times, cycle_time)

# Create the DataFrame from the stations data
df_stations = format_output_as_dataframe(stations)

# Print the DataFrame
total_idle_time = sum(df_stations['Idle time'])  # Calculate total idle time
print(f'Total Idle Time: {total_idle_time}\n')
df_stations



Total Idle Time: 8



,Station,Tasks,Idle time
0,1,1,1
1,2,"2, 3",1
2,3,6,1
3,4,"4, 7, 5, 9",1
4,5,"8, 10",4
5,6,"11, 12",0
